## CRISPR screen analysis

The __Clustered Regularly Interspaced Short Palindromic Repeats (CRISPR) system__ is a bacterial immune system that has been modified for genome engineering. CRISPR consists of two components: a guide RNA (gRNA) and a non-specific CRISPR-associated endonuclease (Cas9). The gRNA is a short synthetic RNA composed of a scaffold sequence necessary for Cas9-binding (trRNA) and ~20 nucleotide spacer or targeting sequence which defines the genomic target to be modified (crRNA). sgRNA (single guide RNA) is a single RNA molecule that contains both the custom-designed short crRNA sequence fused to the scaffold tracrRNA sequence. Cas9 induces double-stranded breaks (DSB) within the target DNA. The resulting DSB is then repaired by either error-prone Non-Homologous End Joining (NHEJ) pathway or less efficient but high-fidelity Homology Directed Repair (HDR) pathway. The NHEJ pathway is the most active repair mechanism and it leads to small nucleotide insertions or deletions (indels) at the DSB site. This results in in-frame amino acid deletions, insertions or frameshift mutations leading to premature stop codons within the open reading frame (ORF) of the targeted gene. The end result is a loss-of-function mutation within the targeted gene.

The ease of generating gRNAs makes CRISPR one of the most scalable genome editing technologies. Genome-wide screens enable systematic targeting of thousands of genes, with one gene targeted per cell, to identify genes driving phenotypes, such as cell survival, drug resistance or sensitivity. 

Below, a __CRISPR perturbation screen analysis to identify genes whose knockout increases cancer cells sensitivity to a drug__. 

CRISPR screen data is analysed using MAGeCK and standard read quality tools. CRISPR screen reads are assessed for quality using __fastp__ sequencing tool. The detection of enriched guides can be performed using MAGeCK. Downstream analysis include visualisations, such as volcano plot, and pathway analysis with tools like fgsea.

### Data

3 samples from human esophageal cancer cell line (OACM5.1): a baseline sample taken at time zero (T0-Control), a sample treated with drug for 8 days (T8-APR-246) and a control sample treated with vehicle for 8 days (T8-Vehicle) [10.1101/2020.11.29.398867](10.1101/2020.11.29.398867).

APR-246 is a mutant-p53 targeted therapeutic (targeting antioxidant pathways) under clinical investigation in acute myeloid leukemia (AML).

### Processing reads 

#### QC and adapter trimming

Use __fastp__ - fast all-in-one preprocessing tool for fastq files which performs quality control, adapter trimming, quality filtering, per-read quality pruning, and provides the filtering results in various formats (HTML, JSON).

In [1]:
import numpy as np
import pandas as pd

In [2]:
samples = ['T0-Control', 'T8-APR-246', 'T8-Vehicle']

adapters = pd.read_csv("adapter_list.tsv", delimiter='\t')
adapters

,Illumina Universal Adapter,AGATCGGAAGAG
0,Illumina Small RNA 3' Adapter,TGGAATTCTCGG
1,Illumina Small RNA 5' Adapter,GATCGTCGGACT
2,Nextera Transposase Sequence,CTGTCTCTTATA
3,CRISPR lentiGuide Puro 5prime,TGTGGAAAGGAC
4,CRISPR lentiGuide Puro 3prime,GTTTTAGAGCTA


The 5' adapter sequence, TGTGGAAAGGAC, from the Brunello library vector used with this dataset (lentiGuide-Puro) will be trimmed.

In [3]:
for sample in samples:
    ! fastp -i {sample}.fastq.gz -o qc/{sample}.fastq.gz --adapter_sequence=TGTGGAAAGGAC -w 4 -h qc/{sample}.html -j qc/{sample}.json

Read1 before filtering:
total reads: 213111
total bases: 15983325
Q20 bases: 14506936(90.7629%)
Q30 bases: 13902387(86.9806%)

Read1 after filtering:
total reads: 1488
total bases: 111505
Q20 bases: 94780(85.0007%)
Q30 bases: 89011(79.8269%)

Filtering result:
reads passed filter: 1488
reads failed due to low quality: 6858
reads failed due to too many N: 0
reads failed due to too short: 204765
reads with adapter trimmed: 211347
bases trimmed due to adapters: 14849002

Duplication rate (may be overestimated since this is SE data): 8.47774%

JSON report: qc/T0-Control.json
HTML report: qc/T0-Control.html

fastp -i T0-Control.fastq.gz -o qc/T0-Control.fastq.gz --adapter_sequence=TGTGGAAAGGAC -w 4 -h qc/T0-Control.html -j qc/T0-Control.json 
fastp v0.23.2, time used: 2 seconds
Read1 before filtering:
total reads: 187963
total bases: 14097225
Q20 bases: 12812053(90.8835%)
Q30 bases: 12284537(87.1415%)

Read1 after filtering:
total reads: 1202
total bases: 90092
Q20 bases: 77306(85.8078%)
Q3

#### Counting

Use __[MAGeCK](10.1186/s13059-014-0554-4)__ (Model-based Analysis of Genome-wide CRISPR/Cas9 Knockout) method for prioritizing single-guide RNAs, genes and pathways in genome-scale CRISPR/Cas9 knockout screens. MAGeCK identifies both positively and negatively selected genes simultaneously, and reports robust results across different experimental conditions. 

To count the number of guides for each gene, [__MAGeCK__](https://sourceforge.net/p/mageck/wiki/usage/) __`count`__ needs a library file containing list of sgRNA names, the sequences and target genes, specifying which guide sequence belongs to which gene. The guides here are from the [Brunello](10.1038/nbt.3437) library which contains 77,441 sgRNAs, an average of 4 sgRNAs per gene, and 1000 non-targeting control sgRNAs. 

In [4]:
sgRNA_library = pd.read_csv("counting/sgRNA_library.tsv", delimiter="\t", header=None)
sgRNA_library.head(10)

,0,1,2
0,ID_1,CATCTTCTTTCACCTGAACG,A1BG
1,ID_2,CTCCGGGGAGAACTCCGGCG,A1BG
2,ID_3,TCTCCATGGTGCATCAGCAC,A1BG
3,ID_4,TGGAAGTCCACTCCACTCAG,A1BG
4,ID_5,ACTGCATCTGTGCAAACGGG,A2M
5,ID_6,ATGTCTCATGAACTACCCTG,A2M
6,ID_7,TGAAATGAAACTTCACACTG,A2M
7,ID_8,TTACTCATATAGGATCCCAA,A2M
8,ID_9,CGGAAGACACAAGGCACCTG,NAT1
9,ID_10,GAACCTTAACATCCATTGTG,NAT1


In [5]:
! mageck count -l "counting/sgRNA_library.tsv" \
--fastq "qc/T0-Control.fastq.gz" "qc/T8-APR-246.fastq.gz" "qc/T8-Vehicle.fastq.gz" \
--sample-label "T0-Control","T8-APR-246","T8-Vehicle" -n output --pdf-report --keep-tmp   --norm-method median 

INFO  @ Thu, 21 Dec 2023 07:39:30: Parameters: /usr/local/bin/mageck count -l counting/sgRNA_library.tsv --fastq qc/T0-Control.fastq.gz qc/T8-APR-246.fastq.gz qc/T8-Vehicle.fastq.gz --sample-label T0-Control,T8-APR-246,T8-Vehicle -n output --pdf-report --keep-tmp --norm-method median 
INFO  @ Thu, 21 Dec 2023 07:39:30: Welcome to MAGeCK v0.5.9.5. Command: count 
INFO  @ Thu, 21 Dec 2023 07:39:30: Loading 77441 predefined sgRNAs. 
WARNING @ Thu, 21 Dec 2023 07:39:30: There are 0 sgRNAs with duplicated sequences. 
INFO  @ Thu, 21 Dec 2023 07:39:30: Parsing FASTQ file qc/T0-Control.fastq.gz... 
INFO  @ Thu, 21 Dec 2023 07:39:30: Determining the trim-5 length of FASTQ file qc/T0-Control.fastq.gz... 
INFO  @ Thu, 21 Dec 2023 07:39:30: Possible gRNA lengths:20 
INFO  @ Thu, 21 Dec 2023 07:39:30: Processing 0M reads ... 
INFO  @ Thu, 21 Dec 2023 07:39:30: Read length:75,63,40,61,54,62 
INFO  @ Thu, 21 Dec 2023 07:39:30: Total tested reads: 1488, mapped: 1016(0.6827956989247311) 
INFO  @ Thu, 

In [6]:
# ! gs -dBATCH -dNOPAUSE -q -dPDFSETTINGS=/prepress -sDEVICE=pdfwrite -sOutputFile=merged.pdf *.pdf

Inspect the MAGeCK count files (sgRNA counts, counts summary and plots pdf) previously generated for the full dataset (as mageck count above was performed on 1% of the full dataset).

__sgRNA counts__

In [7]:
sgRNA_counts = pd.read_csv("counting/full_mageck_sgrna_counts.tsv", delimiter="\t")
sgRNA_counts.head(10)

,sgRNA,Gene,T8-APR-246,T8-Vehicle,T0-Control
0,ID_30030,DUSP14,172,195,205
1,ID_21657,HIST1H2BL,84,55,85
2,ID_69143,CTAGE6,118,232,257
3,ID_70279,TBC1D10C,272,232,308
4,ID_21824,PIP5K1B,352,388,395
5,ID_77019,controlguide578,445,365,293
6,ID_49661,GALNT12,243,213,224
7,ID_23709,MAP7,273,181,242
8,ID_27699,APBB3,96,114,159
9,ID_41971,EXD2,172,121,152


__Counts summary__

Counts summary file summarizes QC measurements of the fastq files, and evaluates the degree of negative selection in known essential genes - only if option --day0-label is provided, MAGeck will run pathway analysis for each sample, and use GSEA metrics (columns 9-13 in table below) to evaluate the quality of the samples.

In [8]:
counts_summary = pd.read_csv("counting/full_mageck_count_summary.tsv", delimiter="\t")
counts_summary

,File,Label,Reads,Mapped,Percentage,TotalsgRNAs,Zerocounts,GiniIndex,NegSelQC,NegSelQCPval,NegSelQCPvalPermutation,NegSelQCPvalPermutationFDR,NegSelQCGene
0,T8-APR-246.fq.gz,T8-APR-246,18165059,14665554,0.8073,77441,2102,0.13620,0,1,1,1,0.0
1,T8-Vehicle.fq.gz,T8-Vehicle,17855968,15019692,0.8412,77441,1659,0.12750,0,1,1,1,0.0
2,T0-Control.fq.gz,T0-Control,20646404,17272052,0.8366,77441,535,0.09077,0,1,1,1,0.0


In [9]:
counts_summary.columns

Index(['File', 'Label', 'Reads', 'Mapped', 'Percentage', 'TotalsgRNAs',
       'Zerocounts', 'GiniIndex', 'NegSelQC', 'NegSelQCPval',
       'NegSelQCPvalPermutation', 'NegSelQCPvalPermutationFDR',
       'NegSelQCGene'],
      dtype='object')

In [10]:
counts_summary_info= {'Column_name': counts_summary.columns, 
                     'Content': ['The fastq file used', 
                                 'The label of that fastq file assigned', 
                                 'Total number reads in the fastq file (Recommended: 100-300 times the number of sgRNAs)', 
                                 'Total number of reads that can be mapped to library',
                                 'Mapped percentage, calculated as Mapped/Reads (Recommended: at least 60%)', 
                                 'Total number of sgRNAs in the library',
                                 'Total number of missing sgRNAs (Recommended: no more than 1%)',
                                 'The Gini Index of the read count distribution. A smaller value indicates more eveness of the count distribution (Recommended: around 0.1 for plasmid or initial state samples, and around 0.2-0.3 for negative selection samples)',
                                 'The enrichment score (ES) of GSEA',
                                 'The p-value of the GSEA analysis (Recommended: smaller than 1e-10)',
                                 'The permutation p-value',
                                 'The FDR of the permutation p-value',
                                 'The number of essential genes found in the library that are evaluated for GSEA analysis'
                                ]}
pd.DataFrame.from_dict(counts_summary_info)

,Column_name,Content
0,File,The fastq file used
1,Label,The label of that fastq file assigned
2,Reads,Total number reads in the fastq file (Recommen...
3,Mapped,Total number of reads that can be mapped to li...
4,Percentage,"Mapped percentage, calculated as Mapped/Reads ..."
5,TotalsgRNAs,Total number of sgRNAs in the library
6,Zerocounts,Total number of missing sgRNAs (Recommended: n...
7,GiniIndex,The Gini Index of the read count distribution....
8,NegSelQC,The enrichment score (ES) of GSEA
9,NegSelQCPval,The p-value of the GSEA analysis (Recommended:...


Data quality is good for the 3 samples. 

- There are enough sequenced reads: for T0 control sample there are 17,272,052 reads mapped to guides, 77,441 guides so we there are ~220 reads per guide (17,272,052/77,441), for T8-APR 14,665,554 reads mapped to guides/77441 guides ~ 190 reads per guide, and for T8-Vehicle 15019692/77441 ~ 190 reads/guide. A minimum of 100 reads per guide, preferably 300, is recommended. OK
- The mapped percentage is good:  >80% mapped for all 3 samples (percentage column). recommended at least 60%. OK
- The sgRNA zero count is good: T0-Control has 0.69% (535/77441 * 100) sgRNAs that have no reads mapped, which is good. The T8 samples are just slightly high at 2.1% (1659/77441 * 100) and 2.7% (2102/77441 * 100). Recommended no more than 1%. 
- The Gini Index is good: Gini index (a measure of inequality) is used in CRISPR analysis to assess if sgRNAs are present in equal amounts. In positive selection experiments, where only some sgRNAs dominate, the index can be high. However, in plasmid library, in early time points, or negative selection experiments, the remaining sgRNAs that haven’t been negatively selected are expected to display a fairly even distribution. A high Gini index in the latter types of sample can indicate CRISPR oligonucleotide synthesis unevenness, low viral transfection efficiency, and overselection, respectively. The Gini Index is 0.09 for T0-Control (initial state) which is good. The T8 samples are higher at 0.12 and 0.13 but good for a negative selection experiment. Recommended: around 0.1 for plasmid or initial state samples, and around 0.2-0.3 for negative selection samples


__Plots PDF__ 

[MAGeCK count report](./counting/full_mageck_count_report.pdf)

In [11]:
from IPython.display import IFrame
IFrame("counting/full_mageck_count_report.pdf", width=600, height=300)

- The __boxplots__ show largely similiar distributions of counts for the 3 samples. The greater length of the box and between whiskers in the T8 samples compared to the control reflect a bit more variability of counts in those samples, more guides with low and high counts in T8 compared to the T0.
- The __distribution of read counts plot__ shows how many guides there are for each count (the frequency sums to the total number of guides 77,441). Similar to the boxplots, the wider distribution for T8 compared to T0 shows that those samples have more guides with low and high counts. The peak for T0 appears lower because it has more points (bins) in the plot.
- The __PCA plot__ shows that the samples from the different conditions separate well. The T8 samples are more similar to each other on PC1 axis than to T0. If the dataset would contain more samples, PCA plots could be used to check for clustering of replicates, batch effect or outliers.
- The __hierarchical clustering plot__ also shows that the T8 samples are more similar to each other than to T0.

### Testing

Use __MAGeCK__ __`test`__ to identify essential genes. Essential means positively or negatively selected sgRNAs and genes. CRISPR positive or negative selection screens can be performed. 

With a positive selection screen, most cells die after the treatment (selection) and it is of interest to identify genes whose sgRNAs increase and dominate, indicating loss of those genes helps cells survive that treatment. This can help identify genes essential for drug resistance. 

With a negative selection screen, most cells survive after the treatment, and it is of interest to identify genes whose sgRNAs decrease (drop out) compared to a control (e.g. vehicle), indicating those genes are needed for the cells to survive with that treatment. This can help identify genes essential for drug sensitivity. 

Regardless of the type of screen performed (positive or negative), MAGeCK can identify both positively and negatively selected genes in the screen by the following algorithm: 
- Raw read counts corresponding to single-guided RNAs (sgRNAs) from different experiments are first normalized using median normalization and mean-variance modeling is used to capture the relationship of mean and variance in replicates.
- The statistical significance of each sgRNA is calculated using the learned mean-variance model.
- Essential genes (both positively and negatively selected) are then identified by looking for genes whose sgRNAs are ranked consistently higher (by significance) using robust rank aggregation (RRA).

__The dataset is from a negative selection screen and the aim is to identify genes whose knockout increases the cancer cells sensitivity to the drug__.

#### Two conditions

__Compare the drug treatment (T8-APR-246) to the vehicle control (T8-Vehicle) using MAGeCK test which employs a robust ranking aggregation (RRA) algorithm__.

In [12]:
! mageck test -k "counting/full_mageck_sgrna_counts.tsv" -t '0' -c '1' -n output_test --normcounts-to-file \
--pdf-report --norm-method median --adjust-method fdr --sort-criteria neg --remove-zero both --gene-lfc-method median

INFO  @ Thu, 21 Dec 2023 07:40:33: Parameters: /usr/local/bin/mageck test -k counting/full_mageck_sgrna_counts.tsv -t 0 -c 1 -n output_test --normcounts-to-file --pdf-report --norm-method median --adjust-method fdr --sort-criteria neg --remove-zero both --gene-lfc-method median 
INFO  @ Thu, 21 Dec 2023 07:40:33: Welcome to MAGeCK v0.5.9.5. Command: test 
INFO  @ Thu, 21 Dec 2023 07:40:33: Loading count table from counting/full_mageck_sgrna_counts.tsv  
INFO  @ Thu, 21 Dec 2023 07:40:33: Processing 1 lines.. 
INFO  @ Thu, 21 Dec 2023 07:40:33: Loaded 77441 records. 
INFO  @ Thu, 21 Dec 2023 07:40:33: Loading Rnw template file: /usr/local/lib/python3.8/site-packages/mageck/test_report.Rmd. 
INFO  @ Thu, 21 Dec 2023 07:40:33: Loading R template file: /usr/local/lib/python3.8/site-packages/mageck/plot_template.RTemplate. 
INFO  @ Thu, 21 Dec 2023 07:40:33: Loading R template file: /usr/local/lib/python3.8/site-packages/mageck/plot_template_indvgene.RTemplate. 
INFO  @ Thu, 21 Dec 2023 07:

Inspect the MAGeCK test files (sgRNA summary, gene summary and PDF report). 

__sgRNA summary__ 

Used to check how the individual guides for the genes of interest performed. 

In [13]:
sgRNA_summary = pd.read_table('testing/output_test.sgrna_summary.txt')
sgRNA_summary.shape

(77441, 15)

In [14]:
sgRNA_summary.head(10)

,sgrna,Gene,control_count,treatment_count,control_mean,treat_mean,LFC,control_var,adj_var,score,p.low,p.high,p.twosided,FDR,high_in_treatment
0,ID_73563,LOC554223,677.86000,5228.600,677.86000,5228.600,2.9455,10355000.0,6690.3000,55.637,1.0,0.000000e+00,0.000000e+00,0.000000e+00,True
1,ID_24663,NRXN1,126.11000,1733.900,126.11000,1733.900,3.7707,1292500.0,883.5100,54.091,1.0,0.000000e+00,0.000000e+00,0.000000e+00,True
2,ID_28115,ACAA2,175.38000,2000.500,175.38000,2000.500,3.5044,1665600.0,1312.3000,50.382,1.0,0.000000e+00,0.000000e+00,0.000000e+00,True
3,ID_38719,AMOTL2,44.33700,829.760,44.33700,829.760,4.1957,308450.0,253.3000,49.350,1.0,0.000000e+00,0.000000e+00,0.000000e+00,True
4,ID_33954,RPL36,0.00000,67.166,0.98527,67.166,5.1017,2255.6,2.9063,38.821,1.0,0.000000e+00,0.000000e+00,0.000000e+00,True
5,ID_24933,EIF4E2,0.98527,66.133,0.98527,66.133,5.0796,2122.1,2.9063,38.215,1.0,1.121381e-319,2.242910e-319,2.894800e-315,True
6,ID_30112,PRSS23,389.18000,2237.200,389.18000,2237.200,2.5201,1707500.0,3424.3000,31.580,1.0,3.517300e-219,7.034600e-219,7.782400e-215,True
7,ID_4797,DLD,0.00000,51.666,0.98527,51.666,4.7295,1334.7,2.9063,29.729,1.0,2.273400e-194,4.546900e-194,4.401400e-190,True
8,ID_75745,PET117,0.00000,50.633,0.98527,50.633,4.7009,1281.8,2.9063,29.123,1.0,1.293100e-186,2.586200e-186,2.225300e-182,True
9,ID_32417,NCAPD3,0.98527,48.566,0.98527,48.566,4.6420,1132.0,2.9063,27.910,1.0,1.391300e-171,2.782700e-171,2.154900e-167,True


In [15]:
sgRNA_summary.columns

Index(['sgrna', 'Gene', 'control_count', 'treatment_count', 'control_mean',
       'treat_mean', 'LFC', 'control_var', 'adj_var', 'score', 'p.low',
       'p.high', 'p.twosided', 'FDR', 'high_in_treatment'],
      dtype='object')

In [16]:
sgRNA_summary_info= {'Column_name': sgRNA_summary.columns, 
                     'Content': ['sgRNA ID', 
                                 'The targeting gene', 
                                 'Normalized read counts in control samples', 
                                 'Normalized read counts in treatment samples',
                                 'Mean read counts in control samples', 
                                 'Mean read counts in treatment samples',
                                 'The log2 fold change of sgRNA',
                                 'The raw variance in control samples',
                                 'The adjusted variance in control samples',
                                 'The score of this sgRNA',
                                 'p-value (lower tail)',
                                 'p-value (higher tail',
                                 'p-value (two-sided)',
                                 'False discovery rate',
                                 'Whether the abundance is higher in treatment samples'
                                ]}
pd.DataFrame.from_dict(sgRNA_summary_info)

,Column_name,Content
0,sgrna,sgRNA ID
1,Gene,The targeting gene
2,control_count,Normalized read counts in control samples
3,treatment_count,Normalized read counts in treatment samples
4,control_mean,Mean read counts in control samples
5,treat_mean,Mean read counts in treatment samples
6,LFC,The log2 fold change of sgRNA
7,control_var,The raw variance in control samples
8,adj_var,The adjusted variance in control samples
9,score,The score of this sgRNA


__Gene summary__ 

The gene summary file contains a row for each gene targeted by sgRNAs ranked by the p.neg field (by default). The values are for both negative and positive selection. There are >20,000 genes in the file for this dataset from a negative selection screen so we negative values are of most interest. 

In [17]:
gene_summary = pd.read_table('testing/output_test.gene_summary.txt')
gene_summary.shape

(20103, 14)

In [18]:
gene_summary.head(10)

,id,num,neg|score,neg|p-value,neg|fdr,neg|rank,neg|goodsgrna,neg|lfc,pos|score,pos|p-value,pos|fdr,pos|rank,pos|goodsgrna,pos|lfc
0,ESD,4,0.000007,0.000019,0.324257,1,4,-1.92830,0.995120,0.99512,0.999775,20004,0,-1.92830
1,MTHFD1L,4,0.000010,0.000032,0.324257,2,4,-0.89778,0.999990,0.99999,0.999991,20103,0,-0.89778
2,SHMT2,4,0.000021,0.000076,0.454208,3,4,-1.13280,0.999980,0.99998,0.999991,20102,0,-1.13280
3,FLI1,4,0.000026,0.000090,0.454208,4,1,-0.08872,0.337410,0.48212,0.999775,9461,1,-0.08872
4,TACC3,4,0.000035,0.000121,0.486139,5,2,-0.75335,0.075059,0.16968,0.998666,3241,2,-0.75335
5,ACYP1,4,0.000079,0.000298,0.767027,6,2,-0.13267,0.413280,0.55952,0.999775,10986,1,-0.13267
6,SYNRG,4,0.000098,0.000369,0.767027,7,3,-1.25300,0.244650,0.38823,0.999603,7600,1,-1.25300
7,KIAA1468,4,0.000110,0.000413,0.767027,8,4,-0.70376,0.999890,0.99990,0.999991,20101,0,-0.70376
8,ZMYND11,3,0.000118,0.000361,0.767027,9,3,-0.78281,0.999880,0.99988,0.999991,20100,0,-0.78281
9,B3GALTL,4,0.000131,0.000498,0.767027,10,3,-0.27607,0.964200,0.96411,0.999775,19342,0,-0.27607


In [19]:
gene_summary.columns

Index(['id', 'num', 'neg|score', 'neg|p-value', 'neg|fdr', 'neg|rank',
       'neg|goodsgrna', 'neg|lfc', 'pos|score', 'pos|p-value', 'pos|fdr',
       'pos|rank', 'pos|goodsgrna', 'pos|lfc'],
      dtype='object')

In [20]:
gene_summary_info = {'Column_name': gene_summary.columns, 
                     'Content': ['Gene ID', 
                                 'The number of targeting sgRNAs for each gene', 
                                 'The RRA lo value of this gene in negative selection', 
                                 'The raw p-value (using permutation) of this gene in negative selection',
                                 'The false discovery rate for this gene in negative selection', 
                                 'The ranking of this gene in negative selection',
                                 'The number of good sgRNA - those whose ranking is below the alpha cutoff (determined by the -gene-test-fdr-threshold option, in negative selection',
                                 'The log2 fold change of this gene in negative selection. Gene lfc calculation is controlled by the -gene-lfc-method option',
                                 'The RRA lo value of this gene in positive selection', 
                                 'The raw p-value (using permutation) of this gene in positive selection',
                                 'The false discovery rate for this gene in positive selection', 
                                 'The ranking of this gene in positive selection',
                                 'The number of good sgRNA - those whose ranking is below the alpha cutoff (determined by the -gene-test-fdr-threshold option, in positive selection',
                                 'The log2 fold change of this gene in positive selection'
                                 ]}
pd.DataFrame.from_dict(gene_summary_info)

,Column_name,Content
0,id,Gene ID
1,num,The number of targeting sgRNAs for each gene
2,neg|score,The RRA lo value of this gene in negative sele...
3,neg|p-value,The raw p-value (using permutation) of this ge...
4,neg|fdr,The false discovery rate for this gene in nega...
5,neg|rank,The ranking of this gene in negative selection
6,neg|goodsgrna,The number of good sgRNA - those whose ranking...
7,neg|lfc,The log2 fold change of this gene in negative ...
8,pos|score,The RRA lo value of this gene in positive sele...
9,pos|p-value,The raw p-value (using permutation) of this ge...


__PDF report__

[MAGeck test report](./testing/output_test.pdf)

In [21]:
from IPython.display import IFrame
IFrame("testing/output_test.pdf", width=600, height=300)

The PDF shows plots of the top 10 negatively and positively selected genes. 
- The distribution plots show the top genes ranked by RRA scores or p values found in the gene summary file. The top 3 negatively selected genes: ESD, MTHFD1L and SHMT2 have similar RRA/p-value scores and are invloved in the glutathione (GSH) pathway and mitochondrial metabolism in determining APR-246 efficacy/sensitivity. APR-246 triggers GSH depletion in cancer cells, and its sensitivity is increased through loss of key enzymes in mitochondrial one-carbon metabolism.
- The plots with the sgRNA counts for the top 10 genes show whether the counts of all the sgRNAs for the top genes are changing similarly. The values are the normalized counts for each sgRNA from the sgRNA summary file. 

#### Visualisation

In addition to the visualisations automatically generated by MAGeCK in the PDF, the output can be further visualised with a volcano plot to plot the magnitude of change for drug treatment versus vehicle control (lfc) versus significance (p-value). As the gene summary file contains two columns for lfc and p-value, one for negative selection and one for positive, these two columns are combined into one for each gene using __awk__. If the `neg|p-value` is smaller than the `pos|p-value` the gene is negatively selected. If the `neg|p-value` is larger than the `pos|p-value` the gene is positively selected. 

In [22]:
! $(which awk) -v FS='	' \
      -v OFS='	' \
      -f scripts/awk.script testing/output_test.gene_summary.txt > testing/output_test.reformatted.gene_summary.txt

In [23]:
! cat scripts/awk.script

# Print new header for first line
NR == 1 { print "gene", "pval", "fdr", "lfc" }

# Only process lines after first
NR > 1 {
    # check if neg pval (column 4) is less than pos pval (column 10)
    if ($4 < $10){
       # if it is, print negative selection values
        print $1, $4, $5, $8
    } else {
       # if it's not, print positive selection values
        print $1, $10, $11, $14
    }
}


In [24]:
gene_summary_trf = pd.read_table('testing/output_test.reformatted.gene_summary.txt')
gene_summary_trf.head(10)

,gene,pval,fdr,lfc
0,ESD,0.000019,0.324257,-1.92830
1,MTHFD1L,0.000032,0.324257,-0.89778
2,SHMT2,0.000076,0.454208,-1.13280
3,FLI1,0.000090,0.454208,-0.08872
4,TACC3,0.000121,0.486139,-0.75335
5,ACYP1,0.000298,0.767027,-0.13267
6,SYNRG,0.000369,0.767027,-1.25300
7,KIAA1468,0.000413,0.767027,-0.70376
8,ZMYND11,0.000361,0.767027,-0.78281
9,B3GALTL,0.000498,0.767027,-0.27607


In [25]:
! Rscript scripts/volcanoplot.rscript --input testing/output_test.reformatted.gene_summary.txt \
--fdr_col 3 --pval_col 2 --lfc_col 4 --label_col 1 --signif_thresh 0.05 --lfc_thresh 0.0 --topn 10

null device 
          1 
[1] "Create volcano_plot.pdf....... DONE."


In [26]:
! cat scripts/volcanoplot.rscript

# Load packages -----------------------------------------------------------

suppressPackageStartupMessages({
    library(dplyr)
    library(ggplot2)
    library(ggrepel)
    library(optparse)    
})

# Fetch arguments  --------------------------------------------------------

option_list = list(
  make_option(c("-i", "--input"), action="store", default=NA, type='character',
              help="input gene summary file"),
  make_option(c("-a", "--fdr_col"), action="store", default=NA, type='integer',
              help="FDR (adjusted p-value) column number"),
  make_option(c("-p", "--pval_col"), action="store", default=NA, type='integer',
              help="pvalue column number"),
  make_option(c("-f", "--lfc_col"), action="store", default=NA, type='integer',
              help="logfc column number"),   
  make_option(c("-l", "--label_col"), action="store", default=NA, type='integer',
              help="labels column number"),  
  make_option(c("-s", "--signif_thresh"), action="store"

[MAGeCK test volcano plot](./testing/volcano_plot.pdf)

In [28]:
from IPython.display import IFrame
IFrame("testing/volcano_plot.pdf", width=600, height=300)

#### Pathway analysis

Use [__fgsea__](https://bioconductor.org/packages/devel/bioc/manuals/fgsea/man/fgsea.pdf) to perform pathway analysis on the results and identify pathways that are changing with the treatment. fgsea needs a ranked list of genes - the list from mageck gene summary result file ranked by `neg|score` - the RRA score column for negative results, and a pathways file in GMT format - the Hallmark pathways from MSigDB.

In [29]:
! cut -f 1,3 testing/output_test.gene_summary.txt > testing/output_test.cut.gene_summary.txt    

In [30]:
pd.read_table('testing/output_test.cut.gene_summary.txt', delimiter='\t').head(10)

,id,neg|score
0,ESD,0.000007
1,MTHFD1L,0.000010
2,SHMT2,0.000021
3,FLI1,0.000026
4,TACC3,0.000035
5,ACYP1,0.000079
6,SYNRG,0.000098
7,KIAA1468,0.000110
8,ZMYND11,0.000118
9,B3GALTL,0.000131


In [31]:
! Rscript scripts/fgsea.R --rnk_file testing/output_test.cut.gene_summary.txt \
--header True --sets_file fgsea/hallmark.pathways.all.v7.4.symbols.gmt --gmt True \
--min_size 15 --max_size 500 --n_perm 1000 --out_tab fgsea/fgsea_pathways.txt \
--plot_opt True --top_num 10 --rda_opt True

Warning message:
package ‘fgsea’ was built under R version 4.3.2 
Warning messages:
1: In preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam,  :
  There are ties in the preranked stats (14.51% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.
2: In preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam,  :
  All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".
[1] "Create fgsea_plots.pdf...... DOne."


In [32]:
! cat scripts/fgsea.R

suppressPackageStartupMessages({
  library(fgsea)
  library(ggplot2)
  library(optparse)
})

option_list <- list(
  make_option(c("-r", "--rnk_file"), type="character", help="Path to ranked genes file"),
  make_option(c("-h", "--header"), type="logical", help = "Does ranked genes file have a header"),
  make_option(c("-s", "--sets_file"), type="character", help = "Path to gene sets file"),
  make_option(c("-g", "--gmt"), type="logical", help = "Is the sets file in GMT format"),
  make_option(c("-o","--out_tab"), type="character", help="Path to output file"),
  make_option(c("-m", "--min_size"), type="integer", help="Minimal size of a gene set to test. All pathways below the threshold are excluded."),
  make_option(c("-x", "--max_size"), type="integer", help="Maximal size of a gene set to test. All pathways above the threshold are excluded."),
  make_option(c("-n", "--n_perm"), type="integer", help="Number of permutations to do. Minimial possible nominal p-value is about 1/nperm"),
  ma

In [33]:
pd.read_table('fgsea/fgsea_pathways.txt', delimiter='\t').head(10)

,pathway,pval,padj,log2err,ES,NES,size,leadingEdge
0,HALLMARK_OXIDATIVE_PHOSPHORYLATION,0.002719,0.078159,0.431708,0.422450,1.217541,184,"CS, PDHB, NDUFS7, NDUFB7, COX7B, NDUFA9, MRPS1..."
1,HALLMARK_UV_RESPONSE_DN,0.003126,0.078159,0.431708,0.430468,1.226789,138,"ABCC1, GCNT1, ADD3, SIPA1L1, NFKB1, SERPINE1, ..."
2,HALLMARK_PEROXISOME,0.016071,0.267852,0.352488,0.431720,1.210982,104,"CRABP2, SLC25A19, FADS1, PEX11A, CADM1, IDH1, ..."
3,HALLMARK_GLYCOLYSIS,0.034965,0.437063,0.241340,0.395600,1.142261,197,"ARPP19, GAL3ST1, P4HA2, HDLBP, B3GAT1, VCAN, I..."
4,HALLMARK_FATTY_ACID_METABOLISM,0.044955,0.449550,0.211400,0.400885,1.147461,155,"PDHB, PPARA, IDH1, HSP90AA1, HSD17B4, FABP1, P..."
5,HALLMARK_REACTIVE_OXYGEN_SPECIES_PATHWAY,0.079920,0.585129,0.155242,0.449017,1.201711,47,"ABCC1, MPO, NDUFS2, CDKN2D, PRDX6, STK25, HMOX..."
6,HALLMARK_APICAL_SURFACE,0.081918,0.585129,0.153159,0.456963,1.217509,43,"LYN, ADAM10, FLOT2, CROCC, APP, NTNG1, LYPD3, ..."
7,HALLMARK_BILE_ACID_METABOLISM,0.105894,0.599401,0.132846,0.399053,1.125537,112,"FADS1, PEX11A, AGXT, IDH1, SLC27A5, GNPAT, HSD..."
8,HALLMARK_IL2_STAT5_SIGNALING,0.107892,0.599401,0.131458,0.381330,1.100535,194,"MXD1, GPR83, NDRG1, WLS, TGM2, CST7, SWAP70, B..."
9,HALLMARK_ADIPOGENESIS,0.147852,0.699301,0.109684,0.375415,1.083243,192,"CS, ANGPT1, NDUFB7, COX7B, GPHN, NKIRAS1, COQ5..."


[FGSEA plots](./fgsea/fgsea_plots.pdf)

In [34]:
from IPython.display import IFrame
IFrame("fgsea/fgsea_plots.pdf", width=600, height=300)

#### Multiple conditions

Use __MAGeck mle__ to compare multiple conditions: the drug treatment (T8-APR-246) to T0-COntrol and the vehicle (T8-Vehicle) to T0-Control.

MAGeCK mle uses a maximum likelihood estimation (MLE) algorithm. It outputs a single value (beta score) per gene instead of a score for both negative and positive selection. A negative beta score indicates negative selection and a positive indicates positive selection. MAGeCK mle can also be used for comparing 2 conditions instead of MAGeCK test (RRA algorithm) but it is slower.

A MAGeCK mle run requires the MAGeCK sgRNA counts and a design matrix. 

Design matrix for 3 conditions: baseline (T0), Vehicle and APR. The 1st row indicates that T0 is baseline sample, the 2nd row indicates Vehicle to T0 comparison, and the 3rd row indicates APR to T0 comparison.

For MAGeCK mle the design matrix:

- must include a header line of condition labels
- the first column is the sample labels that must match labels in read count file
- the second column must be a “baseline” column that sets all values to “1”
- the value in the design matrix is either “0” or “1”
- must have at least one sample of “initial state” (e.g., time 0) that has only one “1” in the corresponding row. That only “1” must be in the baseline column
- the initial state (baseline) sample must be the 1st sample listed, under the header row.

In [35]:
design_matrix = pd.read_csv("mle/full_mageck_mle_design_matrix.tsv", delimiter="\t")
design_matrix

,Samples,baseline,Vehicle,APR
0,T0-Control,1,0,0
1,T8-Vehicle,1,1,0
2,T8-APR-246,1,0,1


In [36]:
! mageck mle -k counting/full_mageck_sgrna_counts.tsv -d mle/full_mageck_mle_design_matrix.tsv \
-n output_mle --norm-method median --threads 4

INFO  @ Thu, 21 Dec 2023 07:44:27: Parameters: /usr/local/bin/mageck mle -k counting/full_mageck_sgrna_counts.tsv -d mle/full_mageck_mle_design_matrix.tsv -n output_mle --norm-method median --threads 4 
INFO  @ Thu, 21 Dec 2023 07:44:27: Cannot parse design matrix as a string; try to parse it as a file name ... 
INFO  @ Thu, 21 Dec 2023 07:44:27: Design matrix: 
INFO  @ Thu, 21 Dec 2023 07:44:27: [[1. 0. 0.] 
INFO  @ Thu, 21 Dec 2023 07:44:27:  [1. 1. 0.] 
INFO  @ Thu, 21 Dec 2023 07:44:27:  [1. 0. 1.]] 
INFO  @ Thu, 21 Dec 2023 07:44:27: Beta labels:baseline,Vehicle,APR 
INFO  @ Thu, 21 Dec 2023 07:44:27: Included samples:T0-Control,T8-Vehicle,T8-APR-246 
INFO  @ Thu, 21 Dec 2023 07:44:27: Loaded samples:T0-Control;T8-Vehicle;T8-APR-246 
INFO  @ Thu, 21 Dec 2023 07:44:27: Sample index: 2;1;0 
INFO  @ Thu, 21 Dec 2023 07:44:27: Loaded 20112 genes. 
INFO  @ Thu, 21 Dec 2023 07:44:28: Final size factor: 0.8688780183350758 1.0565381424478189 1.1107589343544333 
INFO  @ Thu, 21 Dec 2023 07

In [37]:
sgRNA_summary_mle = pd.read_table('mle/output_mle.sgrna_summary.txt')
sgRNA_summary_mle.shape

(77441, 3)

In [38]:
sgRNA_summary_mle.head(10)

,Gene,sgRNA,eff
0,DUSP14,ID_30030,1
1,DUSP14,ID_30029,1
2,DUSP14,ID_30028,1
3,DUSP14,ID_30027,1
4,HIST1H2BL,ID_21657,1
5,HIST1H2BL,ID_21660,1
6,HIST1H2BL,ID_21658,1
7,HIST1H2BL,ID_21659,1
8,CTAGE6,ID_69143,1
9,CTAGE6,ID_69142,1


In [39]:
gene_summary_mle = pd.read_table('mle/output_mle.gene_summary.txt')
gene_summary_mle.shape

(20112, 14)

In [40]:
gene_summary_mle.head(10)

,Gene,sgRNA,Vehicle|beta,Vehicle|z,Vehicle|p-value,Vehicle|fdr,Vehicle|wald-p-value,Vehicle|wald-fdr,APR|beta,APR|z,APR|p-value,APR|fdr,APR|wald-p-value,APR|wald-fdr
0,DUSP14,4,-0.046593,-0.406180,0.81056,0.99079,0.68461,0.96053,0.071544,0.62381,0.78187,0.98870,0.53275,0.84452
1,HIST1H2BL,4,-0.387780,-1.370100,0.09825,0.64533,0.17065,0.95209,-0.066854,-0.23705,0.73523,0.98462,0.81262,0.94193
2,CTAGE6,4,0.070307,0.389960,0.70744,0.98061,0.69656,0.96074,-0.107730,-0.59718,0.60800,0.97167,0.55039,0.85088
3,TBC1D10C,4,-0.004010,-0.053744,0.98086,0.99824,0.95714,0.99509,-0.045206,-0.60519,0.80842,0.99177,0.54505,0.84882
4,PIP5K1B,4,0.190520,1.373800,0.29410,0.87983,0.16950,0.95209,0.179410,1.29340,0.43541,0.93838,0.19587,0.75718
5,controlguide578,1,0.415150,0.391540,0.22817,0.83798,0.69540,0.96074,0.662850,0.62516,0.11182,0.67497,0.53187,0.84402
6,GALNT12,4,0.111720,0.582340,0.54196,0.95750,0.56034,0.95300,0.212830,1.10950,0.34596,0.90939,0.26723,0.76828
7,MAP7,4,0.246180,1.049700,0.17363,0.78962,0.29388,0.95209,0.342050,1.45860,0.12197,0.68885,0.14468,0.75718
8,APBB3,4,-0.043763,-0.338840,0.82244,0.99079,0.73473,0.96644,-0.106320,-0.82249,0.61088,0.97209,0.41080,0.81037
9,EXD2,4,0.151600,0.782790,0.40528,0.92816,0.43375,0.95209,0.225530,1.16470,0.31678,0.89732,0.24414,0.76466


In [41]:
gene_summary_mle.columns

Index(['Gene', 'sgRNA', 'Vehicle|beta', 'Vehicle|z', 'Vehicle|p-value',
       'Vehicle|fdr', 'Vehicle|wald-p-value', 'Vehicle|wald-fdr', 'APR|beta',
       'APR|z', 'APR|p-value', 'APR|fdr', 'APR|wald-p-value', 'APR|wald-fdr'],
      dtype='object')

In [42]:
gene_summary_mle_info = {'Column_name': gene_summary_mle.columns, 
                     'Content': ['Gene ID', 
                                 'The number of targeting sgRNAs for each gene', 
                                 'The beta score of this gene in Vehicle condition', 
                                 'The raw p-value (using permutation) of this gene in Vehicle condition',
                                 'The false discovery rate for this gene in Vehicle selection', 
                                 'The z-score associated with Wald test in Vehicle selection',
                                 'The p-value using Wald test in the Vehicle selection',
                                 'The false discovery rate of the Wald test in the Vehicle selection',
                                 'The beta score of this gene in APR condition', 
                                 'The raw p-value (using permutation) of this gene in APR condition',
                                 'The false discovery rate for this gene in APR selection', 
                                 'The z-score associated with Wald test in APR selection',
                                 'The p-value using Wald test in the APR selection',
                                 'The false discovery rate of the Wald test in the APR selection'
                                 ]}
pd.DataFrame.from_dict(gene_summary_mle_info)

,Column_name,Content
0,Gene,Gene ID
1,sgRNA,The number of targeting sgRNAs for each gene
2,Vehicle|beta,The beta score of this gene in Vehicle condition
3,Vehicle|z,The raw p-value (using permutation) of this ge...
4,Vehicle|p-value,The false discovery rate for this gene in Vehi...
5,Vehicle|fdr,The z-score associated with Wald test in Vehic...
6,Vehicle|wald-p-value,The p-value using Wald test in the Vehicle sel...
7,Vehicle|wald-fdr,The false discovery rate of the Wald test in t...
8,APR|beta,The beta score of this gene in APR condition
9,APR|z,The raw p-value (using permutation) of this ge...


#### Visualize mle results

Create volcano plot from mle gene summary to visualize the result for APT vs T0 using `Gene` column for label option, `APR|beta` for log fold change, `APR|wald-p-value` for p-value, and `APR|wald-fdr` for fdr option.

In [43]:
! Rscript scripts/volcanoplot.rscript --input mle/output_mle.gene_summary.txt \
--fdr_col 14 --pval_col 13 --lfc_col 9 --label_col 1 --signif_thresh 0.05 --lfc_thresh 0.0 --topn 10

null device 
          1 
[1] "Create volcano_plot.pdf....... DONE."


[MAGeCK mle APT vs T0 volcano plot](./mle/volcano_plot_APR-T0.pdf)

In [44]:
from IPython.display import IFrame
IFrame("mle/volcano_plot_APR-T0.pdf", width=600, height=300)

Create volcano plot from mle gene summary to visualize the result for Vehicle vs T0 using `Gene` column for label option, `Vehicle|beta` for log fold change, `Vehicle|wald-p-value` for p-value, and `Vehicle|wald-fdr` for fdr option.

In [45]:
! Rscript scripts/volcanoplot.rscript --input mle/output_mle.gene_summary.txt \
--fdr_col 8 --pval_col 7 --lfc_col 3 --label_col 1 --signif_thresh 0.05 --lfc_thresh 0.0 --topn 10

null device 
          1 
[1] "Create volcano_plot.pdf....... DONE."


[MAGeCK mle Vehicle vs T0 volcano plot](./mle/volcano_plot_Vehicle-T0.pdf)

In [46]:
from IPython.display import IFrame
IFrame("mle/volcano_plot_Vehicle-T0.pdf", width=600, height=300)